In [1]:
### Importing all the neccessary packages ###
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import intake
import seasonality.seasonalityfunctions as sf

In [138]:
### Allows us to use dask to speed up some calculations ###
from dask.distributed import Client
client = Client()


In [139]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63665,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63679,Total threads: 2
Dashboard: http://127.0.0.1:63681/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:63668,


In [137]:
catalog_url = 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json'
col = intake.open_esm_datastore(catalog_url)
col

,unique
variable,78
long_name,75
component,5
experiment,4
frequency,6
vertical_levels,3
spatial_domain,5
units,25
start_time,12
end_time,13


In [141]:
col_subset = col.search(frequency=["daily"], component="atm", variable="PRECT",
                        experiment=["20C"])

In [142]:
dsets = col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dsets.keys()}")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency'


/var/folders/m4/3slgbrxj3z3dm65l82539j6w0000gq/T/ipykernel_15643/234168810.py:1: DeprecationWarning: cdf_kwargs and zarr_kwargs are deprecated and will be removed in a future version. Please use xarray_open_kwargs instead.
  dsets = col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})



Dataset dictionary keys:
 dict_keys(['atm.HIST.daily', 'atm.20C.daily', 'atm.RCP85.daily'])


In [143]:

ds_20C = dsets['atm.20C.daily']['PRECT']



In [147]:
test = ds_20C.sel(lat=46,lon=-105+360,member_id=105, method="nearest").compute()

In [3]:
dr = pd.date_range('1920-01-01','2005-12-31', freq='1D')
dates_noleap = dr[(dr.day != 29) | (dr.month != 2)]
test['time'] = dates_noleap

In [4]:
# Renaming datarray to match existing code and changeing from m/s to m
ds_subset = test.rename({'PRECT':'precip'})*86400.0

In [5]:
### Changing data to mm and removing leap days ###
precip = ds_subset['precip']*1000.0
precip = precip.sel(time=~((precip.time.dt.month == 2) & (precip.time.dt.day == 29)))

### Calculating the mean precipitation for each grid point
annual_mean_precip = precip.mean(dim='time')

### Calculating the mean annual cycle ###
annual_precip_cycle= precip.groupby('time.dayofyear').mean(dim='time')

# Pandas creates leap day again with dayofyear use here
annual_precip_cycle = annual_precip_cycle[annual_precip_cycle.dayofyear != 60]
annual_precip_cycle['dayofyear'] = np.arange(1,365+1,1)

annual_precip_cycle_var = precip.groupby('time.dayofyear').std(dim='time')

# Pandas creates leap day again with dayofyear use here
annual_precip_cycle_var = annual_precip_cycle_var[annual_precip_cycle_var.dayofyear != 60]
annual_precip_cycle_var['dayofyear'] = np.arange(1,365+1,1)

### Create Daily precip from hourly ###
##daily_precip = precip.resample(time='1D').sum()

### Create The annual daily precip anomalies ###
annual_means = precip.groupby('time.year').mean(dim='time')


anomalies = precip.groupby('time.year') - annual_means
#anomalies = anomalies_withleap.sel(time=~((anomalies_withleap.time.dt.month == 2) & (anomalies_withleap.time.dt.day == 29)))

climDailyMeanAnomaly = annual_precip_cycle - annual_mean_precip

In [ ]:
#TODO write code to regenerate the other test datasets

In [288]:
np.savez_compressed("5th_order_smooth_of_apc", sf.harmonic_smoothing(annual_precip_cycle,num_harm=5))

In [162]:

np.savez_compressed("precip_1d_test_data_mm",precip = precip)